In [132]:
from tkinter import *
from tkinter import ttk
import serial
import matplotlib.pyplot as plt
ser = serial.Serial('COM3',  baudrate=115200, timeout=1)

In [139]:
r=ser.readline() 
r

b'31153 106180\r\n'

In [62]:
r.decode()

'31754 107293\r\n'

In [63]:
r.decode().split()

['31754', '107293']

In [65]:
a,b=r.decode().split()


print(a)

31754


In [73]:
INdata=ser.readline() 
INdata

b'30620 103466\r\n'

In [96]:
INdata=ser.readline() 
REDdata,IRdata=INdata.decode().split()
print(INdata)
print(REDdata)
print(IRdata)

b'31629 109364\r\n'
31629
109364


In [100]:
INdata = 0
REDdata = [0]*200
IRdata = [0]*200

In [102]:


for i in range(199):
    REDdata[i] = REDdata[i+1] 
    IRdata[i] = IRdata[i+1]         
INdata=ser.readline() 
REDdata[199],IRdata[199]=INdata.decode().split()

    
print(REDdata[198],REDdata[199])

ValueError: not enough values to unpack (expected 2, got 0)

In [140]:
from tkinter import *
from tkinter import ttk
import serial
import matplotlib.pyplot as plt
ser = serial.Serial('COM3',  baudrate=115200, timeout=1)

In [141]:
root = Tk()
root.geometry('820x820')
root['bg']="#333"
ca = Canvas(root, width=800, height=800)
ca.place(x=8,y=8)

timestr=StringVar()
lb=Label(root,textvariable=timestr)
lb.place(x=20,y=20,height=30)
style=ttk.Style(root)
style.configure(".",font="Arial 15")

dd = [0]*200
outdata = [0]*198

def tt():
    global dd
    global outdata
    newdd = [0]*199
    
    ca.delete("all")
    
    for i in range(199):
        dd[i] = dd[i+1]
    dd[199] = int(ser.readline())
      
    ddAverage=0
    for i in range(199):
        ddAverage += dd[i]
    ddAverage/=200
    ddAverage = 800 - ( (ddAverage-70000) / 3 )
    
    ddCorrection = 600 - ddAverage
      
    for i in range(199):
        newdd[i] = 800 - ( (dd[i]-70000) / 3 ) + ddCorrection
    for i in range(198):
        outdata[i] = ( newdd[i] + newdd[i+1] )/2

    print( outdata[197] )
    #print( "-----------" )

    #----------------------------------------
    
    x=0
    pxmax=0
    pmax=0
    pxmin=0
    pmin=65535
    plv=100
    dr=1
    ax,ay=[],[]

    for p in outdata:
        if dr==1:
            if p>pmax:
                pmax=p
                pxmax=x
            else:
                d=pmax-p
                if d > plv:
                    #print(x,pxmax,pmax)
                    dr=0      
                    ax+=[pxmax]
                    ay+=[pmax]
                    pmin=pmax
        else:
            if p<pmin:
                pmin=p
                pxmin=x     
            else:
                d=p-pmin
                if d > plv:
                    #print(x,pxmin,pmin)
                    pmax=pmin
                    dr=1
        x+=1
    #print( ax )
    #print( "-----------" )
    
    #----------------------------------------
    
    dx=[]
    for i in range( len(ax)-1 ):
        dx += [ ax[i+1]-ax[i] ] 
    
    sumdx = sum(dx)
    if sumdx==0:
        sumdx=1      
    
    lendx = len(dx)
    if lendx==0:
        lendx=1    
      
    PPG = round(60*50 / (sumdx/lendx) )
    if PPG==3000:
        PPG=0  
    #print(PPG)  
    timestr.set(PPG)
    
    #----------------------------------------    
       
    xy=[]
    for i in range(len(outdata)):
        xy += ( 800-i*6 , outdata[i] )           
    ca.create_line(xy, width=2, fill='blue')   
    
    #---------------------------------------- 
    
    ca.after(1, tt)

ca.after(1, tt)

mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-141-e9c52aeb169d>", line 25, in tt
    dd[199] = int(ser.readline())
ValueError: invalid literal for int() with base 10: b'127 0\r\n'
